# Using MoviePy for Good (and Evil\*) 

## Helper Functions

In [35]:
from IPython.display import HTML

def display_video(url, width=500, height=500):
    return HTML("""
    <video width="%d" height="%d" style="margin:auto auto;" autoplay loop="true" >
      <source src="%s" type="video/mp4">
    </video>
    """ % (width, height, url))

## Data Visualizations

You can use MoviePy for data visualizations. Here's an example by [Zulko](http://zulko.github.io/blog/2014/11/29/data-animations-with-python-and-moviepy/).

In [31]:
import matplotlib.pyplot as plt
import numpy as np
from moviepy.video.io.bindings import mplfig_to_npimage
import moviepy.editor as mpy

# DRAW A FIGURE WITH MATPLOTLIB

duration = 2

fig_mpl, ax = plt.subplots(1,figsize=(5,3), facecolor='white')
xx = np.linspace(-2,2,200) # the x vector
zz = lambda d: np.sinc(xx**2)+np.sin(xx+d) # the (changing) z vector
ax.set_title("Elevation in y=0")
ax.set_ylim(-1.5,2.5)
line, = ax.plot(xx, zz(0), lw=3)

# ANIMATE WITH MOVIEPY (UPDATE THE CURVE FOR EACH t). MAKE A GIF.

def make_frame_mpl(t):
    line.set_ydata( zz(2*np.pi*t/duration))  # <= Update the curve
    return mplfig_to_npimage(fig_mpl) # RGB image of the figure

animation =mpy.VideoClip(make_frame_mpl, duration=duration)
animation.write_videofile("./sinc_mpl.mp4", fps=60)

display_video('sinc_mpl.mp4', 512, 384)

[MoviePy] >>>> Building video ./sinc_mpl.mp4
[MoviePy] Writing video ./sinc_mpl.mp4


 99%|█████████▉| 120/121 [00:03<00:00, 40.30it/s]

[MoviePy] Done.
[MoviePy] >>>> Video ready: ./sinc_mpl.mp4 



## Using PyGame

Sometimes you need somthing more advanced than rendering a mask or text on a video. As long as you can generate an 2D NumPy array of colors, you can anything!

Here's an example using PyGame:

In [32]:
import os
import pygame
import moviepy.editor as mpy
from IPython.display import HTML

# Headless SDL
os.environ['SDL_VIDEODRIVER'] = 'dummy'

# Start up pygame
pygame.init()

# Going to hide the main display (make it small)
pygame.display.set_mode((1,1))

# Buffer we're going to draw changes in
screen = pygame.Surface((1024, 768), pygame.SRCALPHA, 32)

# Get your own duck
duckface = pygame.image.load("res/duck.bmp")
position = duckface.get_rect()

# Function to get a frame 
# Returns a NumPy-style array size WIDTHxHEIGHTx3
# Represeting a 2D RGB field. (Each component is 8-bit)

def next_frame(dt):
    global position # globals, lol

    # Translate by (2, 0)
    position = position.move(2, 0)

    # Fill the screen with black
    screen.fill(0x00000000)

    # Draw duckface
    screen.blit(duckface, position)

    # Bring the image from the other buffer to 
    # main
    pygame.display.flip()

    # For some reason, we have to do this or
    # else the image comes out flipped
    newscreen = pygame.transform.rotate(screen, 90)

    # Reverse the columns of the pixel array
    # We could do it with MoviePy with .fx(vfx.mirror_y)
    return pygame.surfarray.array3d(newscreen)[:,::-1]
    

duration = 1 # in seconds
animation = mpy.VideoClip(next_frame, duration=duration)
animation.write_videofile("./video.mp4", fps=24) 


display_video('video.mp4', 256, 192)


[MoviePy] >>>> Building video ./video.mp4
[MoviePy] Writing video ./video.mp4


 96%|█████████▌| 24/25 [00:01<00:00, 12.69it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./video.mp4 



## Making Contemporary Video (GIF Compositing)


In [33]:
HTML("""
<center><video width="300" height="300" autoplay loop="true" >
  <source src="./res/datboi.mp4" type="video/mp4">
</video> &nbsp;&nbsp;&nbsp; <video width="300" height="300" autoplay loop="true" >
  <source src="./res/sunset.mp4" type="video/mp4">
</video></center>
""")

In [34]:
# Import everything needed to edit video clips
from moviepy.editor import *


# Because it's a transparent GIF, it has a mask.
clip = VideoFileClip("res/datboi.gif", has_mask=True).subclip(0,.3)
sunset = VideoFileClip("res/sunset.mp4").subclip(0,.3)

video = CompositeVideoClip([sunset, clip])

# Write the result to a file
video.write_videofile("datboi_over_sunset.mp4",fps=24)

display_video('datboi_over_sunset.mp4')

[MoviePy] >>>> Building video datboi_over_sunset.mp4
[MoviePy] Writing video datboi_over_sunset.mp4


100%|██████████| 8/8 [00:00<00:00, 42.70it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: datboi_over_sunset.mp4 



## Dependencies

I've only tested this on macOS. Please update the instructions if you find yourself installing this on other platforms. Install with `pip install -r requirements.txt`